# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
# TODO: Implement load the data here.

# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = "traffic-sign-data/train.p"
testing_file = "traffic-sign-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
import numpy as np

In [3]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [4]:
X_train_original = X_train
X_test_original = X_test

In [5]:
# TODO: Implement data normalization here.

X_train_standardised = (X_train - X_train.mean()) / np.std(X_train)
X_test_standardised = (X_test - X_test.mean()) / np.std(X_test)
X_train_normalised = (X_train - X_train.mean()) / (np.max(X_train) - np.min(X_train))
X_test_normalised = (X_test - X_test.mean()) / (np.max(X_test) - np.min(X_test))

In [6]:
print("Min: %d" % np.min(X_train_normalised))
print("Max: %d" % np.max(X_train_normalised))
print("Mean: %d" % np.mean(X_train_normalised))

Min: 0
Max: 0
Mean: 0


## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [7]:
import keras

Using TensorFlow backend.


In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

In [9]:
# TODO: Build a two-layer feedforward neural network with Keras here.
# Sequential model is a linear stack of layers.
model = Sequential()
# Dense is a fully connnected layer
# Fully connected layer
# -> allows for non-linear combination of features 
# discovered in the previous layer
model.add(Dense(512, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Layer 2
hidden1 = model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Convert to 43 labels since 43 outputs
model.add(Dense(43))
# Softmax to compute probabilities
output = model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           1573376     dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 128)           65664       dropout_1[0][0]                  
___________________________________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [9]:
# One-hot encode labels
y_train = np_utils.to_categorical(y_train,nb_classes=43)
y_test = np_utils.to_categorical(y_test, nb_classes=43)

In [10]:
# Normalised input
X_train = X_train_normalised
X_test = X_test_normalised

In [11]:
# Reshape input
X_train = X_train.reshape(len(X_train), 32*32*3)
X_test = X_test.reshape(len(X_test), 32*32*3)

In [13]:
batch_size = 100
nb_epoch = 100

In [18]:
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, y_test))

# STOP: Do not change the tests below. Your implementation should pass these tests.
# assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']


Train on 39209 samples, validate on 12630 samples
Epoch 1/2
39209/39209 [==============================] - 14s - loss: 1.8461 - acc: 0.5054 - val_loss: 1.0850 - val_acc: 0.6807
Epoch 2/2
39209/39209 [==============================] - 14s - loss: 0.7222 - acc: 0.7822 - val_loss: 0.7391 - val_acc: 0.7880


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(\
    X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# TODO: Compile and train the model to measure validation accuracy.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=2,
                    verbose=1, validation_data=(X_test, y_test))

In [43]:
print("X_train: ", len(X_train))
print("y_train: ", len(y_train))
print("X_val: ", len(X_val))
print("y_val: ", len(y_val))

X_train:  31367
y_train:  31367
X_val:  7842
y_val:  7842


**Validation Accuracy**: (fill in here)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [48]:
y_train[0]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [ ]:
y_val = np_utils.to_categorical(y_val, 43)

In [54]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

model = Sequential()
# Layer 1: Conv
model.add(Convolution2D(32, 
                        kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))

# Layer 2:
model.add(Flatten())
model.add(Dense(512, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Layer 3
hidden1 = model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Convert to 43 labels since 43 outputs
model.add(Dense(43))
# Softmax to compute probabilities
output = model.add(Activation('softmax'))

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 145s - loss: 1.0289 - acc: 0.7180 - val_loss: 0.2400 - val_acc: 0.9396
Epoch 2/2
31367/31367 [==============================] - 143s - loss: 0.2480 - acc: 0.9306 - val_loss: 0.1246 - val_acc: 0.9654


In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

**Validation Accuracy**: 0.9654 after two epochs.
```
Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 145s - loss: 1.0289 - acc: 0.7180 - val_loss: 0.2400 - val_acc: 0.9396
Epoch 2/2
31367/31367 [==============================] - 143s - loss: 0.2480 - acc: 0.9306 - val_loss: 0.1246 - val_acc: 0.9654
```

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [55]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

model = Sequential()
# Layer 1: Conv
model.add(Convolution2D(32, 
                        kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

# Layer 2:
model.add(Flatten())
model.add(Dense(512, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Layer 3
hidden1 = model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Convert to 43 labels since 43 outputs
model.add(Dense(43))
# Softmax to compute probabilities
output = model.add(Activation('softmax'))

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 42s - loss: 1.3786 - acc: 0.6184 - val_loss: 0.3510 - val_acc: 0.9050
Epoch 2/2
31367/31367 [==============================] - 40s - loss: 0.3183 - acc: 0.9102 - val_loss: 0.1366 - val_acc: 0.9651


**Validation Accuracy**: 0.9651

```
Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 42s - loss: 1.3786 - acc: 0.6184 - val_loss: 0.3510 - val_acc: 0.9050
Epoch 2/2
31367/31367 [==============================] - 40s - loss: 0.3183 - acc: 0.9102 - val_loss: 0.1366 - val_acc: 0.9651
```

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [27]:
# Convnet parameters
nb_filters = 32
kernel_size = (3, 3)
input_shape = (32, 32, 3)
pool_size = (2, 2)
dropout_conv = 0.2
dropout_fc = 0.5

In [56]:
# TODO: Re-construct the network and add dropout after the pooling layer.

model = Sequential()
# Layer 1: Conv
model.add(Convolution2D(nb_filters, 
                        kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))

# Layer 2:
model.add(Flatten())
model.add(Dense(512, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Layer 3
hidden1 = model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Convert to 43 labels since 43 outputs
model.add(Dense(43))
# Softmax to compute probabilities
output = model.add(Activation('softmax'))

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 43s - loss: 1.4900 - acc: 0.5787 - val_loss: 0.3572 - val_acc: 0.9081
Epoch 2/2
31367/31367 [==============================] - 45s - loss: 0.4357 - acc: 0.8719 - val_loss: 0.1841 - val_acc: 0.9570


**Validation Accuracy**: 0.9570 after two epochs.

```
Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 43s - loss: 1.4900 - acc: 0.5787 - val_loss: 0.3572 - val_acc: 0.9081
Epoch 2/2
31367/31367 [==============================] - 45s - loss: 0.4357 - acc: 0.8719 - val_loss: 0.1841 - val_acc: 0.9570
```

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.